# Training Data Cleaning and Split

In [3]:
import os
import sys
import logging
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Fix this later, imports should work without this
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))


from common.utils import collect_files, get_or_create_folder
from common.logger import get_logger_config
from common.constants import (
    BASE_RAW_DATA_DIR,
    BASE_LOGS_DIR,
    BASE_PLOTS_DIR,
    BASE_REPORTS_CSV_DIR,
)

## Remove entries with `precursor_charge` less than 2

## Check how do the peptides in the dataset here overlap the peptides from Kevin

In [6]:
from common.utils import get_or_create_folder, collect_files

In [11]:
peptides_file_paths = collect_files(BASE_REPORTS_CSV_DIR, ext="csv")


In [15]:
# TODO: Improve the `collect_files` function
peptides_file_paths = [path for path in peptides_file_paths if "unique_modified_peptides" in path or "unique_peptides" in path]

assert peptides_file_paths, peptides_file_paths
# Grab all csv of interest but ATTENTION;
# loading all many csv files will increase the computation time
df = pd.concat([pd.read_csv(file) for file in peptides_file_paths], ignore_index=True)
df.head(20)

['/home/hjisaac/AI4Science/instanovo_instadeep/InstanovoGlyco/reports/csv/PXD026629/unique_peptides.csv', '/home/hjisaac/AI4Science/instanovo_instadeep/InstanovoGlyco/reports/csv/PXD026629/unique_modified_peptides.csv', '/home/hjisaac/AI4Science/instanovo_instadeep/InstanovoGlyco/reports/csv/PXD031032/unique_peptides.csv', '/home/hjisaac/AI4Science/instanovo_instadeep/InstanovoGlyco/reports/csv/PXD031032/unique_modified_peptides.csv', '/home/hjisaac/AI4Science/instanovo_instadeep/InstanovoGlyco/reports/csv/PXD031025/unique_peptides.csv', '/home/hjisaac/AI4Science/instanovo_instadeep/InstanovoGlyco/reports/csv/PXD031025/unique_modified_peptides.csv', '/home/hjisaac/AI4Science/instanovo_instadeep/InstanovoGlyco/reports/csv/PXD044641_PXD035158/unique_peptides.csv', '/home/hjisaac/AI4Science/instanovo_instadeep/InstanovoGlyco/reports/csv/PXD044641_PXD035158/unique_modified_peptides.csv', '/home/hjisaac/AI4Science/instanovo_instadeep/InstanovoGlyco/reports/csv/PXD026649/unique_peptides.csv'

,Unique Peptides,Unique Modified Peptides
0,HNGTGGR,NaN
1,SQNCHNSSSR,NaN
2,AAGMNHTK,NaN
3,ANASHDQPQK,NaN
4,HNDSGASECR,NaN
5,GGGGGGGGGGGGGSGSSSGSSTSR,NaN
6,RQQQQQQQQQQQQK,NaN
7,QQQQQQQQQQQQK,NaN
8,KNDSGAYR,NaN
9,KCLNHTTQK,NaN
